In [ ]:
!conda install -c conda-forge tslearn

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.spatial.distance as dist
from scipy import fftpack as fp
from scipy import linalg

from tslearn.clustering import KShape
from tslearn.datasets import CachedDatasets
from tslearn.preprocessing import TimeSeriesScalerMeanVariance

In [ ]:
seed = 0
np.random.seed(seed)
X_train, y_train, X_test, y_test = CachedDatasets().load_dataset("Trace")
# Keep first 3 classes and 50 first time series
X_train = X_train[y_train < 4]
X_train = X_train[:50]
np.random.shuffle(X_train)
# For this method to operate properly, prior scaling is required
X_train = TimeSeriesScalerMeanVariance().fit_transform(X_train)
sz = X_train.shape[1]

# kShape clustering
ks = KShape(n_clusters=3, verbose=True, random_state=seed)
y_pred = ks.fit_predict(X_train)

plt.figure()
for yi in range(3):
    plt.subplot(3, 1, 1 + yi)
    for xx in X_train[y_pred == yi]:
        plt.plot(xx.ravel(), "k-", alpha=.2)
    plt.plot(ks.cluster_centers_[yi].ravel(), "r-")
    plt.xlim(0, sz)
    plt.ylim(-4, 4)
    plt.title("Cluster %d" % (yi + 1))

plt.tight_layout()
plt.show()

In [ ]:
X_train = TimeSeriesScalerMeanVariance().fit_transform(X_train)
X_train = X_train.reshape(50, 275)
#X_train = pd.DataFrame(X_train)
X_train.shape

In [ ]:
# fuzzy c-meansの定義
def fcm(X, C, THETA, max_iter):
    plt.figure(figsize=(16, 12))
    n, m = X.shape
    # メンバシップ値の初期化
    u = np.random.rand(C, n)
    u /= u.sum(axis=0)

    step = 0
    # for step in range(5):
    while step < max_iter:
        step += 1
        # u^θを先に計算しておく
        u_theta = u**THETA
        # クラスタ中心の計算
        b = u_theta @ X / u_theta.sum(1)[:, np.newaxis]
        # メンバシップ行列の更新
        d = dist.cdist(b, X)

        u = d**(-2/(THETA-1.0))
        u /= u.sum(axis=0)

        vpc = (u**2).sum() / n

        # 目的関数の計算
        obj = ((u**THETA) * d**2).sum()

        if abs(obj_prev - obj) < 1e-5:
            break
        else:
            obj_prev = obj

    for yi in range(C):
        plt.subplot(C, 1, 1 + yi)
        for xx in range(n):
            plt.plot(X_train[xx], "k-", alpha=u[yi, xx])
        plt.plot(b[yi], "r-")
        plt.xlim(0, b.shape[1])
        plt.ylim(-4, 4)
        plt.title("Cluster %d" % (yi + 1))

    return u, b, d, step, vpc, obj


# fuzzy c-shape+の定義
def fcs_plus(X, C, THETA, max_iter):
    plt.figure(figsize=(16, 12))
    n, m = X.shape
    # メンバシップ値の初期化
    u = np.random.rand(C, n)
    u /= u.sum(axis=0)

    step = 0
    obj_prev = np.inf
    # for step in range(5):
    while step < max_iter:
        step += 1
        # u^θを先に計算しておく
        u_theta = u**THETA
        # クラスタ中心の計算
        b = u_theta @ X / u_theta.sum(1)[:, np.newaxis]
        # メンバシップ行列の更新
        #d = dist.cdist(b, X)
        d = np.array([[get_SBD(b[j], X[i]) for i in range(n)] for j in range(C)])

        u = d**(-2/(THETA-1.0))
        u /= u.sum(axis=0)

        vpc = (u**2).sum() / n

        # 目的関数の計算
        obj = ((u**THETA) * d**2).sum()

        if abs(obj_prev - obj) < 1e-5:
            break
        else:
            obj_prev = obj

    for yi in range(C):
        plt.subplot(C, 1, 1 + yi)
        for xx in range(n):
            plt.plot(X_train[xx], "k-", alpha=u[yi, xx])
        plt.plot(b[yi], "r-")
        plt.xlim(0, b.shape[1])
        plt.ylim(-4, 4)
        plt.title("Cluster %d" % (yi + 1))

    return u, b, d, step, vpc, obj


# Function for getting a shape-based distance (SBD)
def get_SBD(x, y):

    # Define FFT-size based on the length of input
    p = int(x.shape[0])
    FFTlen = int(2**np.ceil(np.log2(2*p-1)))

    # Compute the normalized cross-correlation function (NCC)
    CC = fp.ifft(fp.fft(x, FFTlen)*fp.fft(y, FFTlen).conjugate()).real

    # Reorder the IFFT result
    CC = np.concatenate((CC[-(p-1):], CC[:p]), axis=0)

    # To avoid zero division
    denom = linalg.norm(x) * linalg.norm(y)
    if denom < 1e-10:
        denom = np.inf
    NCC = CC / denom

    # Search for the argument to maximize NCC
    ndx = np.argmax(NCC, axis=0)
    dist = 1 - NCC[ndx]

    return dist

In [ ]:
u, b, d = fcm(X_train, C=3, THETA=2, max_iter=200)
u.shape, b.shape, d.shape, X_train.shape

In [ ]:
distortions = []
for c in range(2, 10):
    u, b, d, s, vpc, sse = fcs_plus(X_train, C=c, THETA=2, max_iter=100)
    vmpc = 1 - c/(c-1)*(1-vpc)
    distortions.append(sse)
    print(c, s, vpc, vmpc, sse)

In [ ]:
plt.plot(range(2, 10), distortions, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')
plt.show()

In [ ]:
u, b, d, s, vpc, sse = fcs_plus(X_train, C=4, THETA=2, max_iter=100)
u.shape, b.shape, d.shape, s, vpc

In [ ]:
pd.DataFrame(u)

In [ ]:
pd.DataFrame(b)

In [ ]:
X_train[0]

In [ ]:
pd.DataFrame(X_train)